## Logistic Regression using Newton Method and KFold

In [9]:
import numpy as np
import pandas as pd

In [10]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
y = dataset.target

target_names = list(dataset.target_names)
print(target_names)

['setosa', 'versicolor', 'virginica']


In [81]:
# Change to binary class
y = (y > 0).astype(int)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [118]:
# Model: Linear Regression
class LogReg:
    """
    This implementation of Logistic Regression uses the Newton's Method for optimization.
    """
    def __init__(self, num_iters=20, tolerance = 1e-10, epsilon = 10e-8, threshold=0.5, verbose=False):
        self.num_iters = num_iters
        self.tolerance = tolerance
        self.epsilon = epsilon # subtracted to make hessian invertible
        self.threshold = threshold
        self.verbose = verbose
        
    def add_ones(self, X):
        return np.concatenate((np.ones((len(X),1)), X), axis = 1)
    
    
    def sigmoid(self, X, theta):
        return 1/(1 + np.exp(X@theta))
    
    def get_hessian_inv(self, X, theta):
        htheta1theta = np.diag(self.sigmoid(X, theta).T@(1-self.sigmoid(X, theta))).reshape(-1, 1)
        hessian = (X.T*htheta1theta)@X
        hessian_inv = np.linalg.inv(hessian + self.epsilon*np.eye(X.T.shape[0]))
        return hessian_inv
    
    def cost(self, X, y_true):
        return np.mean((X@self.theta - y_true)**2)
    
    def fit(self, X, y):
        X = X.copy()
        X = self.add_ones(X)
        y = y.reshape(-1, 1)
        
        self.theta = np.zeros((len(X[0]), 1))
        
        current_iter = 1
        norm = 1
        while (norm >= self.tolerance and current_iter < self.num_iters):
            old_theta = self.theta.copy()
            grad = -X.T@(y - self.sigmoid(X, self.theta))
            grad= grad.reshape(-1, 1)
            hessian_inv = self.get_hessian_inv(X, self.theta)
            
            self.theta = self.theta - hessian_inv@grad
            
            if self.verbose:
                print(f'cost for {current_iter} iteration : {self.cost(X, y)}')
            norm = np.linalg.norm(old_theta - self.theta)
            current_iter += 1
            
        return self.theta
    
    def evaluate(self, X, y):
        """
        Returns mse loss for a dataset evaluated on the hypothesis
        """
        X = self.add_ones(X)
        return self.cost(X, y)
    
    def predict(self, X):
        prob = self.predict_proba(X)
        return (prob > self.threshold).astype(int)
        
    def predict_proba(self, X):
        """
        Returns probability of predictions.
        """
        X = self.add_ones(X)  
        return self.sigmoid(X, self.theta)

In [119]:
logreg = LogReg()

In [120]:
logreg.fit(X, y)

array([[ 0.20655598],
       [-0.03572809],
       [-0.13137099],
       [ 0.12153157],
       [ 0.03110443]])

In [121]:
predictions = logreg.predict(X)

In [122]:
np.sum(y == predictions) / len(y)

66.66666666666667

In [123]:
class KFoldCrossVal:
    """
    Performs k-fold cross validation on each combination of hyperparameter set
    
    Input
    ............
    X : Features (m, n)
    y : target (m, 1)
    hyperparameter_Set : Dictionary of hyperparameters for k-fold
    num_of_folds: Number of folds, k; default:10
    verbose: Checks whether to print parameters on every iteration; Boolean; Default: False
    """
    def __init__(self, hyperparameter_set, num_of_folds=10, verbose=True):
        self.hyperparameter_set = hyperparameter_set
        self.k = num_of_folds
        self.verbose = verbose
    
    import sys
    if ('numpy' not in sys.modules) and ('np' not in dir()): #import numpy if not done
        import numpy as np
    
    #def get_model_no(self):
        
        
    def shuffle_data(self, X, y):
        shuffle_arr = np.random.permutation(len(X))
        X_shuffled = X[shuffle_arr]
        y_shuffled = y[shuffle_arr].reshape(-1, 1)
        
        return X_shuffled, y_shuffled
    
    def get_kfold_arr_index(self, subset_size, last_index):
        array_indexes = [0]
        for fold_no in range(self.k):
            if fold_no != (self.k-1):
                array_indexes.append((fold_no+1)*subset_size)
            elif fold_no == (self.k - 1): # To accomodate examples not part of the 
                array_indexes.append(last_index) #for last index
        return array_indexes
    
    def get_split_data_fold(self, X, y, array_indexes, fold_no):
        start = array_indexes[fold_no]
        end = array_indexes[fold_no+1]
        X_val = X[start: end]
        y_val = y[start: end]
        
        X_train = np.delete(X, [start,end], axis=0)
        y_train = np.delete(y, [start,end]).reshape(-1,1)
        
        return X_train, y_train, X_val, y_val
    
    def get_hyperparameter_sets(self, hyperparameter_dict):   
        """
        Converts the hyperparameter dictionary into all possible combinations of hyperparameters

        Return
        ..............
        Array of hyperparameter set
        """
        import itertools

        parameter_keys = hyperparameter_dict.keys()
        parameter_values = hyperparameter_dict.values()

        parameter_array = []
        for params in itertools.product(*parameter_values):
            parameter_array.append(params)

        parameter_sets = []
        for parameter_values in parameter_array:
            parameter_set = dict(zip(parameter_keys, parameter_values))
            parameter_sets.append(parameter_set)

        return parameter_sets
    
    def evaluate(self, X, y):
        models = self.get_hyperparameter_sets(self.hyperparameter_set)
        
        #print(f'Performing k-fold for {len(models)} models and {len(models) * self.k} cross validations' )
        m = len(X)
        
        generalization_mse = []
        X, y = self.shuffle_data(X, y)
        subset_size = int(m/self.k)
        
        array_indexes = self.get_kfold_arr_index(subset_size, m+1)
        
        for hyperparameters in models:
            model = LogReg(**hyperparameters)
            fold_mse_arr = []
            for fold_no in range(self.k - 1):
                X_train, y_train, X_val, y_val = self.get_split_data_fold(X, y, array_indexes, fold_no)
                model.fit(X_train, y_train)
                mse = model.evaluate(X_val, y_val)
                fold_mse_arr.append(mse)
            cv_mse = np.mean(fold_mse_arr)
            if self.verbose:
                print(f'{hyperparameters}, mse: {cv_mse}')
            generalization_mse.append(cv_mse)
            
        lowest_gen_mse_index = np.argmin(generalization_mse)
        lowest_mse = generalization_mse[lowest_gen_mse_index]
        best_model = models[lowest_gen_mse_index]
        
        return lowest_mse, best_model

In [124]:
hyp = {
    'num_iters': [1, 2, 3, 4],
    'tolerance': [1e-3, 1e-5, 1e-7, 1e-10],
    'epsilon': [10e-6, 10e-8, 10e-10],
    'threshold': [0.45, 0.5]
}

kcv = KFoldCrossVal(hyp, 5, verbose=False)

In [125]:
kcv.evaluate(X, y)

(0.6302374734075014,
 {'num_iters': 4, 'tolerance': 0.001, 'epsilon': 1e-09, 'threshold': 0.45})